## Ссылки
1. [Напряжённость гравитационного поля (Вики)](https://ru.wikipedia.org/wiki/Напряжённость_гравитационного_поля)

# 18. Гравитационное поле Земли
Вычислить и построить графики зависимости напряженности гравитационного поля Земли от радиус-вектора (начало в центре Земли). Визуализировать векторное поле.

In [1]:
G = 6.67428e-11 # G - Гравитационная постоянная
M = 5.9742e+24  # M - Масса земли, Кг

In [2]:
# Напряжённость на расстоянии R
def E(R: float) -> float:
    global G, M
    return G * M / R**2

In [4]:
E(6371000)

9.823561527927778